### Intialisierung

In [1]:
#Load needed modules
import os

from genai_core_azure_openai_session import AzureOpenAISession
from langchain.llms.openai import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings

import sklearn.metrics.pairwise 
from langchain.document_loaders.pdf import PDFMinerLoader

import pandas as pd
import numpy as np

import json

from textwrap import wrap

import urllib3
urllib3.disable_warnings()

from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

In [27]:
# Initialize Azure Open AI Session

AzureOpenAISession.class_init(llm_gw_api_key=os.environ.get("LLM_GW_API_KEY"))

gpt4 = AzureChatOpenAI(
    openai_api_key=AzureOpenAISession._azure_open_ai_api_key,
    openai_organization=AzureOpenAISession._azure_open_ai_org,
    openai_api_base=AzureOpenAISession._api_base,
    openai_proxy=AzureOpenAISession._proxy,
    openai_api_version="2023-05-15",
    openai_api_type="azure",
    deployment_name="gpt-4", #"gpt-35-turbo-16k",
    model="gpt-4",#"gpt-35-turbo-16k",
#    model_version="vision-preview",
    temperature=0)

openAI_embedding_engine = OpenAIEmbeddings(
    openai_api_key=AzureOpenAISession._azure_open_ai_api_key,
    openai_organization=AzureOpenAISession._azure_open_ai_org,
    openai_api_base=AzureOpenAISession._api_base,
    openai_proxy=AzureOpenAISession._proxy,
    openai_api_version="2023-05-15",
    openai_api_type="azure",
    model="text-embedding-ada-002")

api base: https://cr-svc-llm-gateway-abalbk37qa-oa.a.run.app/azure-openai
proxy: 


In [4]:
#Some helper functions
def get_embedding(obj, engine):
    if type(obj) is list:
        return engine.embed_documents(obj)
    else: 
        return engine.embed_documents([obj])[0]

def cosine_similarity(a, b):
    return sklearn.metrics.pairwise.cosine_similarity([a], [b])[0][0]

def pretty_print(str):
    [print("\n".join(wrap(each, width=120))) for each in str.splitlines()]

# **LLM Joyride durch die Versicherungswelt**

## **Open the Hood - Semantic Search**

<img src="data/open-the-hood-wide.jpg" alt="image" width="800" height="auto">

### **Wieso ist semantische Suche für Gen AI Lösungen relevant?**

Um sicherzustellen, dass ein großes Sprachmodell nicht in Halluzinationen gerät, empfiehlt es sich, alle erforderlichen Fakten in der Anfrage bereitzustellen. Wenn beispielsweise eine Anfrage zu einem AXA-Versicherungsprodukt gestellt wird, sollten relevante Produktinformationen wie AVBs der Anfrage beigelegt werden.

Der Ansatz, relevante Fakten mittels semantischer Suche zu ermitteln und in die Generierung von Antworten einzubeziehen, wird als [Retrieval Augmented Generation (RAG)](https://docs.aws.amazon.com/sagemaker/latest/dg/jumpstart-foundation-models-customize-rag.html) 


<img src="data/rag.jpg" alt="image" width="600" height="auto">

### **Was ist semantische Suche**?

Ein konkretes Beispiel aus dem Versicherungsbereich: Angenommen, ich möchte herausfinden, welche Klauseln in den "Allgemeinen Vertragsbedingungen MF" sich auf Elektroauto-Batterien beziehen. Bei der Suche innerhalb der AVBs ist eine herkömmliche Textsuche wenig effektiv: Soll ich nach "Akku" oder nach "Batterie" suchen?

Die Darstellung von Wörtern als Zeichenkette ermöglicht keinen direkten Vergleich zwischen den Bedeutungen verschiedener Wörter.
Suchbegriff|Begriff in AVB|Vergleich Zeichen für Zeichen
---|---|:---:
Batterie|Batterie| <font color="green"> ✔ </font>
Akku|Batterie|❌
Baterie|Batterie|❌
Auto|Batterie|❌

Semantische Suche erlaubt es, nach der **Bedeutung** eines Worts zu suchen:
Suchbegriff|Begriff in AVB|Semantischer Vergleich
---|---|:---:
Batterie|Batterie|<font size="3"> ⬤ </font>
Akku|Batterie | <font size="4"> ◕ </font>
Baterie|Batterie | <font size="4"> ◕ </font>
Auto | Batterie | <font size="4"> ◔ </font>


Ein Platzhalter für die Bedeutung eines Wortes (oder eines ganzen Satzes) sind sogenannte "Embeddings". 
Large Language Modelle wie Chat GPT stellen (quasi als Nebenprodukt) eine Abbildung von Wörtern als Embeddings zur Verfügung.

### **Was sind Embeddings?**



* Ein Embedding ist ein Repräsentant für die Bedeutung eines Worts (oder eines ganzes Satzes)
* Ein Embedding ist ein n-dimensionaler Vektor (n>>3)
* Embeddings lassen sich vergleichen (was dem Vergleich der Bedeutung des Wortes (resp. Satzes) entspricht 😎)
* Tomas Mikolov, Google, 2013: Skip Gram Model / Word2Vec

### **Was sind die Eigenschaften von Embeddings**


* Wörter mit ähnlicher Bedeutung (wie "Batterie" und "Akku") haben ähnliche Embeddings (resp. Embedding-Vektoren)
* D.h. die Embedding-Vektoren von Worten mit ähnlicher Bedeutung weisen in die selbe Richtung
* Cosine Similarity ... ist das meist verwendete Mass für die Aehnlichkeit zweier Embeddings. Es misst den Winkel zwischen den Embedding-Vektoren (genauer der Cosinus des Winkels).
* Cosine Similarity lässt sich sehr effizient berechnen:
$cos \varphi = {{\vec a \cdot \vec b} \over {|\vec a| \cdot |\vec b|}}$

<img src="data/cosine-similarity.jpg" alt="image" width="400" height="auto">

#### Beispiel:

##### Embedding-Vektoren für "Batterie", "Akku" und "Auto" vom OpenAI-Modell holen

In [ ]:
batterie = get_embedding("Ich fahre einen Tesla", engine=openAI_embedding_engine)
akku = get_embedding("Akku", engine=openAI_embedding_engine)
auto = get_embedding("Auto", engine=openAI_embedding_engine)

In [ ]:
np.linalg.norm(batterie)

##### Aehnlichkeit der Worte vergleichen

In [ ]:
cosine_similarity(batterie, akku)

In [ ]:
cosine_similarity(batterie, auto)

## **Cruising on AXA's freeway**

<img src="data/mustang-cruising-wide.jpg" alt="image" width="800" height="auto">

Unsere "Ausstattung" für die Ausfahrt:
* GenAI Core Framework (inkl. GPT 4)
* [Allg. Vertragsbedingungen MF](data/MF-AVB.pdf)
* [MF-Police von G. Dolge](data/MF-Police.pdf)
* [Offerte von Smile für G. Dolge](data/MF-Offerte-Smile.pdf)


### Demo 1: Suche relevante AVB-Passagen

#### Embeddings für AVBs aufbauen

In [5]:
# AVBs laden und Embedding-Vektoren erstellen
AVBs = pd.read_csv("data/MF-AVB.csv", sep="@")
AVBs["embedding"] = get_embedding(AVBs["Text"].tolist(), openAI_embedding_engine)

#### Funktion für semantische Suche auf AVBs

In [6]:
#1. Berechne Embedding vom Suchtext
#2. Berechne Similarität zwischen AVB-Klauseln und Suchtext
#2. Gib die AVB-Klauseln mit höchster "Similarität" zurück

def suche_relevante_AVBs(such_text, AVBs, max_avbs=10):
    such_text_embedding = get_embedding(such_text, engine=openAI_embedding_engine)
    AVBs["similarity"] = AVBs["embedding"].apply(lambda avb_embedding: cosine_similarity(avb_embedding, such_text_embedding))
    return AVBs.sort_values(by="similarity", ascending=False)[:max_avbs]

#### In AVBs nach "Elektroauto" suchen und relevanteste AVB-Passagen anzeigen

In [7]:
such_text = "Elektroauto"
suche_relevante_AVBs(such_text, AVBs)[["id", "Teil", "Titel", "Untertitel", "similarity"]]

,id,Teil,Titel,Untertitel,similarity
98,D5.1,Services und Zusatzleistungen,E-Mobilität Ladestation,Versicherte Sache,0.811702
102,D6.1,Services und Zusatzleistungen,E-Mobilität Batterie,Versicherte Sache,0.805950
100,D5.3,Services und Zusatzleistungen,E-Mobilität Ladestation,Nicht versichert sind:,0.795725
103,D6.2,Services und Zusatzleistungen,E-Mobilität Batterie,Versicherungsschutz,0.795004
99,D5.2,Services und Zusatzleistungen,E-Mobilität Ladestation,Versicherungsschutz,0.794113
30,A12.5,Rahmenbedingungen des Versicherungsvertrags,Schadenfall,E-Mobilität Ladestation / E-Mobilität Batterie,0.792154
101,D5.4,Services und Zusatzleistungen,E-Mobilität Ladestation,Leistungen im Schadenfall,0.790991
104,D6.3,Services und Zusatzleistungen,E-Mobilität Batterie,Leistung im Schadenfall,0.785303
73,C9.4,Kaskoversicherung: Schäden an Ihrem Fahrzeug,Versichertes Fahrzeug und Zubehör,NaN,0.784164
105,D6.4,Services und Zusatzleistungen,E-Mobilität Batterie,Besondere Leistungen,0.783118


In [8]:
# funktioniert auch für ganze Sätze!
such_text = "Ich hatte einen Zusammenstoss mit einer Wildsau"
suche_relevante_AVBs(such_text, AVBs)[["id", "Teil", "Titel", "Untertitel", "similarity"]]

,id,Teil,Titel,Untertitel,similarity
62,C2.7,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Kollision mit Tieren,0.808831
28,A12.3,Rahmenbedingungen des Versicherungsvertrags,Schadenfall,Kasko,0.803845
59,C2.4,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Naturereignisse,0.800500
112,D8.1,Services und Zusatzleistungen,Verletzung an Ihnen und Mitfahrenden,Versicherungsschutz,0.792004
61,C2.6,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Schäden durch Marder und Nagetiere,0.791725
88,D1,Services und Zusatzleistungen,Grobfahrlässigkeit,NaN,0.788643
63,C2.8,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Böswillige Beschädigung,0.787401
57,C2.2,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Veruntreuung,0.785362
56,C2.1,Kaskoversicherung: Schäden an Ihrem Fahrzeug,"Schäden durch Natur, Tiere\nund Unbekannte (Te...",Diebstahl inkl. Car-Hacking,0.784880
34,A12.8,Rahmenbedingungen des Versicherungsvertrags,Schadenfall,Angetrunkener und fahrunfähiger Zustand oder k...,0.784003


### Demo 2: Deckungsprüfung auf Basis Police und AVB

#### Police laden

In [9]:
with open('data/MF-Police.json', 'r') as openfile:
    police = json.load(openfile)

#### Funktion zur Deckungsprüfung


In [10]:
#1. Finde diejenigen AVB-Klauseln, die auf den Schadenhergang am besten "passen"
#2. Übergib die Anfrage, obige AVBs, die Police und den Schadenhergang an Chat GPT

def pruefe_deckung(vertrag, AVBs, schadenhergang, prompt, max_avbs=15):
    relevante_avbs = suche_relevante_AVBs(schadenhergang, AVBs, max_avbs=max_avbs)["Text"]
    relevante_avbs = "\n\n".join(relevante_avbs.to_list())
    return gpt4.predict(prompt.format(avbs=relevante_avbs, vertrag=str(vertrag), schaden=schaden))

In [11]:
prompt ="Kläre, ob und wie der beschriebene Schadenhergang gedeckt ist.\n\
\n\
\n\n\n\
Vertrag: ''' {vertrag} ''' \n\n\n\
Vertragsbedingungen: ''' {avbs} '''\n\n\n\
Schadenhergang: ''' {schaden} '''"

#### Deckung prüfen

In [28]:
schaden = "Ich bin am Hallauer Bergrennen von der Strasse gekommen."
pretty_print(pruefe_deckung(police, AVBs, schaden, prompt))

Der beschriebene Schadenhergang ist nicht durch die Motorfahrzeugversicherung "OPTIMA" gedeckt. Gemäß den
Vertragsbedingungen B6.2 und C11.3 besteht kein Versicherungsschutz für Schäden, die bei der Teilnahme an Rennen,
Rallyes und ähnlichen Wettfahrten sowie auf Fahrten auf Renn- und Trainingsstrecken entstehen. Da der Schaden während
eines Bergrennens entstanden ist, fällt dieser unter diese Ausschlussklausel.


In [ ]:
schaden = "Ich war auf der N1 von Winterthur nach St. Gallen unterwegs. Auf Höhe Wil stand plötzlich ein Reh auf der Fahrbahn. Ich konnte nicht mehr bremsen und es kam zum Zusammenstoss"
pretty_print(pruefe_deckung(police, AVBs, schaden, prompt))

In [ ]:
schaden = "Ich habe in der Migros Rosenberg in der Tiefgarage parkiert. Als ich wieder zum Auto zurück kam, entdeckte ich eine Beule, die wohl in meiner Abwesenheit durch ein anderes Fahrzeug verursacht worden ist."
pretty_print(pruefe_deckung(police, AVBs, schaden, prompt))

### Demo 3: Offertvergleich

#### Offerte von Smile laden

In [ ]:
with open('data/MF-Offerte-Smile.json', 'r') as openfile:
    offerte = json.load(openfile)

#### Offerte und Police vergleichen

In [ ]:
prompt ="Vergleiche den Selbstbehalt der Police von AXA mit dem Angbot von Smile.\n\
\n\
\n\
Police AXA: ''' {vertrag} ''' \n\n\n\
Offerte Smile: ''' {offerte} '''"

pretty_print(gpt4.predict(prompt.format(vertrag=str(police), offerte=str(offerte))))

In [ ]:
prompt ="Vergleiche die Nettoprämie der Police von AXA mit dem Angbot von Smile.\n\
\n\
\n\
Police AXA: ''' {vertrag} ''' \n\n\n\
Offerte Smile: ''' {offerte} '''"

pretty_print(gpt4.predict(prompt.format(vertrag=str(police), offerte=str(offerte))))